In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from pandas.tseries.offsets import DateOffset

### Uncomment each csv_import at a time to create a forecast for different bedroom size

# csv_import = "City_Zhvi_1bedroom.csv"
#csv_import = "City_Zhvi_2bedroom.csv"
# csv_import = "City_Zhvi_3bedroom.csv"
csv_import = "City_Zhvi_4bedroom.csv"
#csv_import = "City_Zhvi_5BedroomOrMore.csv"
# csv_import = 'State_Zhvi_1bedroom.csv'
# csv_import = 'State_Zhvi_2bedroom.csv'
# csv_import = 'State_Zhvi_3bedroom.csv'
# csv_import = 'State_Zhvi_4bedroom.csv'
# csv_import = 'State_Zhvi_5BedroomOrMore.csv'

data = pd.read_csv(csv_import, header = None)

data.at[0,0] = 0

data = data.dropna(axis=0)

data_refined = data.iloc[:,3:]

state_list = data_refined[3].tolist()[1:]

data_refined = data.iloc[:,9:].T

df = data_refined.T.reset_index(drop=True).T

for index, state in enumerate(state_list):
    df.rename(columns = {index + 1 : state}, inplace = True)
    
df = df.rename(columns = {0: "Date"})

df['New York'] = pd.to_numeric(df['New York'])
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace = True)
model = sm.tsa.statespace.SARIMAX(df['New York'], order = (0, 1, 1), seasonal_order = (1, 1, 0, 12))
model_fit = model.fit()
future_dates = [df.index[-1] + DateOffset(months = x) for x in range(0,60)]
future_datest_df = pd.DataFrame(index = future_dates[1:], columns = df.columns)
future_df = pd.concat([df,future_datest_df])
future_df['New York_forecast'] = model_fit.predict(start = 291, end = 351, dynamic = True)
# future_df[['CA', 'CA_forecast']].plot(figsize=(12, 8))

for state in state_list[1:]:
    a = pd.DataFrame(df[state])
    if len(a.columns) == 1:
        df[state] = pd.to_numeric(df[state])
        model = sm.tsa.statespace.SARIMAX(df[state], order = (0, 1, 1), seasonal_order = (1, 1, 0, 12))
        model_fit = model.fit()
        future_df[state + "_forecast"] = model_fit.predict(start = 291, end = 351, dynamic = True)

future_df.T.to_csv("forecast_" + csv_import)

C:\Users\Jun Jeon\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (1,2,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246

LinAlgError: LU decomposition error.